In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import re
import os
from math import isnan
from datetime import datetime
from socket import gethostname
import string
from random import *
import cx_Oracle

host = os.environ['ORACLE_HOST']
port = os.environ['ORACLE_PORT']
servname = os.environ['ORACLE_SERVNAME']
passwrd = os.environ['ORACLE_PASS']
user = os.environ['ORACLE_USER']

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        return conn

In [2]:
dsn_tns = cx_Oracle.makedsn(host,port,service_name=servname)
conn = cx_Oracle.connect(user,passwrd,dsn_tns)
cr = conn.cursor()

In [16]:
con = create_connection(r"gilsontsl.db")

2.6.0


In [74]:
con_gilson = create_connection(r"/Volumes/npsg/tecan/SourceData/SecondStage/Scripts/gilsontsl.db")

2.6.0


In [27]:
con.close()

In [10]:
TABLE_NAME = 'GILSON_SET_TABLE'

df = pd.read_sql_query(f"SELECT * from {TABLE_NAME}", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

                     RUN_DATE          ID  SAMPLE_NUMBER SAMPLE_NAME  \
0  2020-04-06 16:47:32.317022  1520030001              0    M21535_5   
1  2020-04-06 16:47:32.343616  1520030001              1    M21551_5   
2  2020-04-06 16:47:32.344137  1520030001              2    M21523_5   
3  2020-04-06 16:47:32.345212  1520030001              3    M24173_2   
4  2020-04-06 16:47:32.346140  1520030001              4    M23131_6   

  TUBE_BARCODE BROOKS_BARCODE  
0   FB02646205            127  
1   FB02646225            127  
2   FB02646185            127  
3   FB03658814            127  
4   FB02699823            127  


In [12]:
pd.read_sql_query('SELECT DISTINCT ID FROM GILSON_SET_TABLE', con)

,ID
0,1520030001
1,1520030002
2,1520030003


In [4]:
d_test = pd.read_csv('test_files/1520030001_secStg_127.tsl',sep='\t')
d_test.head()

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127.0,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127.0,2,P01S02


In [5]:
col_names = [re.search(r'^.+?(?=\[)', c).group(0) for c in d_test.columns]
col_names

['SkipPause',
 'MethodName',
 'SampleName',
 'SampleAmount',
 'SampleDescription',
 'SampleId',
 'InjectionYesNo',
 'PeakInformation',
 'SampleInjection',
 'NOTES_STRING',
 '#Sample Well',
 '#Plate_Sample']

In [7]:
def convert_caps_underscore(string):
    prefix_tup_int = [m.span(0) for m in re.finditer(r'^.+(?=[A-Z])', string)]
    suffix_tup_int = [m.span(0) for m in re.finditer(r'[A-Z](?:[^A-Z]*$)', string)]
    #print(prefix_tup_int, suffix_tup_int)
    p_start = prefix_tup_int[0][0]
    p_end = prefix_tup_int[0][1]
    s_start = suffix_tup_int[0][0]
    s_end = suffix_tup_int[0][1]
    return f'{string[p_start:p_end].upper()}_{string[s_start:s_end].upper()}'

In [23]:
sqlite_create_table_query = '''CREATE TABLE GILSON_TSL_TABLE (NUMBER TEXT NOT NULL,RUN_DATE TIMESTAMP,PROJECT_ID TEXT NOT NULL,PLATE_ID TEXT NOT NULL,'''

In [24]:
comma = ','
reformat_col_names = ['NUMBER', 'RUN_DATE', 'PROJECT_ID', 'PLATE_ID']
for i,c in enumerate(col_names):
    #if i == len(col_names) - 1:
    #    comma = ''
    if ord(c[0]) < 65:
        c = c[1:]
        if ' ' in c:
            prefix, suffix = c.split(' ')
            c = prefix[0].upper() + prefix[1:].lower() + suffix[0].upper() +suffix[1:].lower()
    if '_' in c:
        prefix, suffix = c.split('_')
        c = prefix[0].upper() + prefix[1:].lower() + suffix[0].upper() +suffix[1:].lower()
    new_c = convert_caps_underscore(c)
    #if 'DESCRIPTION' in new_c:
    #    text_type = 'PRIMARY KEY'
    #else:
    text_type = 'TEXT NOT NULL'
    sqlite_create_table_query += f'{new_c} {text_type}{comma}'
    reformat_col_names.append(new_c)
    
    
sqlite_create_table_query += 'GILSON_NUM TEXT NOT NULL);'
reformat_col_names.append('GILSON_NUM')

In [30]:
sqlite_create_table_query

'CREATE TABLE GILSON_TSL_TABLE (NUMBER TEXT NOT NULL,RUN_DATE TIMESTAMP,PROJECT_ID TEXT NOT NULL,PLATE_ID TEXT NOT NULL,SKIP_PAUSE TEXT NOT NULL,METHOD_NAME TEXT NOT NULL,SAMPLE_NAME TEXT NOT NULL,SAMPLE_AMOUNT TEXT NOT NULL,SAMPLE_DESCRIPTION TEXT NOT NULL,SAMPLE_ID TEXT NOT NULL,INJECTIONYES_NO TEXT NOT NULL,PEAK_INFORMATION TEXT NOT NULL,SAMPLE_INJECTION TEXT NOT NULL,NOTES_STRING TEXT NOT NULL,SAMPLE_WELL TEXT NOT NULL,PLATE_SAMPLE TEXT NOT NULL,GILSON_NUM TEXT NOT NULL);'

In [128]:
reformat_col_names

['NUMBER',
 'RUN_DATE',
 'PROJECT_ID',
 'PLATE_ID',
 'SKIP_PAUSE',
 'METHOD_NAME',
 'SAMPLE_NAME',
 'SAMPLE_AMOUNT',
 'SAMPLE_DESCRIPTION',
 'SAMPLE_ID',
 'INJECTIONYES_NO',
 'PEAK_INFORMATION',
 'SAMPLE_INJECTION',
 'NOTES_STRING',
 'SAMPLE_WELL',
 'PLATE_SAMPLE',
 'GILSON_NUM']

In [63]:
con_gilson.execute(sqlite_create_table_query)

In [53]:
sqlite_delete_table_query = '''DROP TABLE GILSON_TSL_TABLE;'''
con_gilson.execute(sql_delete_query)

In [73]:
con_gilson.commit()
con_gilson.close()

In [34]:
con_gilson.execute(sqlite_delete_table_query)

In [71]:
con_gilson.execute("DELETE FROM GILSON_TSL_TABLE where PROJECT_ID = 15200600")

In [72]:
pd.read_sql_query("SELECT * from GILSON_TSL_TABLE where PROJECT_ID = 15200600", con_gilson)

,NUMBER,RUN_DATE,PROJECT_ID,PLATE_ID,SKIP_PAUSE,METHOD_NAME,SAMPLE_NAME,SAMPLE_AMOUNT,SAMPLE_DESCRIPTION,SAMPLE_ID,INJECTIONYES_NO,PEAK_INFORMATION,SAMPLE_INJECTION,NOTES_STRING,SAMPLE_WELL,PLATE_SAMPLE,GILSON_NUM


In [57]:
sql_delete_query = """DELETE from GILSON_TSL_TABLE""" 
con_gilson.execute(sql_delete_query)

In [75]:
df = pd.read_sql_query('SELECT * FROM GILSON_TSL_TABLE', con_gilson)

In [76]:
df.to_csv('test_files/export_db.csv')

In [47]:
dir_fi = [fi for fi in os.listdir('test_files/')]
dir_fi

['1520060002_secStg_15200600020.tsl',
 '1520030001_secStg_127.tsl',
 '1520030002_secStg_134.tsl',
 'gilson_set_file_127.txt',
 'gilson_set_file_134.txt',
 '1520030003_secStg_138.tsl',
 'gilson_set_file_138.txt',
 'Excel_gilson_script_test.xlsx',
 '1520060001_secStg_15200600010.tsl']

In [9]:
def grab_rows(fi_content_df):
    row_idx=[]
    for i in range(fi_content_df.shape[0]):
        try:
            if pd.isna(fi_content_df['#Plate_Sample[True,115,13]'][i]):
                pass#print('nothing')
            else:
                row_idx.append(i)
        except TypeError:
            print(i)
    return row_idx

In [8]:
def blank_barcode():
    characters = string.ascii_letters + string.punctuation  + string.digits
    blank_bc =  "BLANK_" + ''.join(choice(characters) for x in range(randint(8, 16)))
    return blank_bc 

In [7]:
def get_col_names_tbl(table_name, con):
    cursor = con.execute(f'SELECT * FROM {table_name}')
    names = list(map(lambda x: x[0], cursor.description))
    return {'length':len(names),'names':[(i,n) for i,n in enumerate(names)]}

In [48]:
def show_query(table_name, con):
    res = con.execute(f"SELECT * FROM {table_name}")
    for r in res:
        print(r)

In [6]:
def grab_colm_idx(col_name, con):
    return [i for i,n in get_col_names_tbl("GILSON_TSL_TABLE", con)['names'] if n == col_name][0]

In [207]:
def split_string_input_file(input_file):
    if '/' in input_file:
        input_file = input_file.split('/')[-1]
    return input_file

def imbue_rows(dir_fi, con, commit=False, gil_num=[gethostname()]):
    nbr = 0 
    # check to see if the file path has '/'
    tmp_dir_fi = split_string_input_file(dir_fi)
    split_fi_name = tmp_dir_fi.split('_')
    projectid = split_fi_name[0]
    # read into pandas df and coerce datatypes to string
    fi_content_df = pd.read_csv(dir_fi, sep='\t',dtype={'NOTES_STRING[False,68,9]': str, \
                                                       'SampleId[False,90,5]' : str})
    # ensure strings for each value
    fi_content_df['SampleId[False,90,5]'] = fi_content_df['SampleId[False,90,5]'].apply(lambda x: 'NaN')
    fi_content_df['SampleAmount[False,78,3]'] = fi_content_df['SampleAmount[False,78,3]'].apply(lambda x: f"{str(x)}")
    fi_content_df['#Sample Well[True,109,10]'] = fi_content_df['#Sample Well[True,109,10]'].apply(lambda x: f"{str(x)}")
    # split string and create new column
    fi_content_df['plateid'] = fi_content_df['NOTES_STRING[False,68,9]'].str.split('/').str[1]
    # drop the original column since we got the projecti from the file already
    fi_content_df.drop('NOTES_STRING[False,68,9]', axis=1, inplace=True)
    current_col_names = fi_content_df.columns.tolist()[:-1]
    # remove plateid colm as separate list
    plateid_list = [x for x in fi_content_df.plateid.tolist() if not pd.isna(x)]
    # slice df to exlucde plateid
    fi_content_df = fi_content_df[current_col_names]
    
    # get the valid row indices (non-flush/std/shutdown)
    row_idx = grab_rows(fi_content_df)
    len_rows = len(row_idx)
    # get length of gilson table to import to
    len_cols = get_col_names_tbl('GILSON_TSL_TABLE', con)['length']
    # if not a full set, add blanks 
    if len_rows % 4 != 0:
        len_rows = len_rows + (4-len_rows%4)
    # the index nbr for the blank sample well
    blank_sample_idx = grab_colm_idx('SAMPLE_WELL', con) 
    # add random blank barcode
    blank_bc_idx = grab_colm_idx('SAMPLE_DESCRIPTION', con) 
    
    for i in range(len_rows): 
        nbr += 1
        row_data = ["BLANK"]*len_cols
        row_data[blank_sample_idx] = nbr
        row_data[2] = projectid
        row_data[0] = nbr
        row_data[-1] = gil_num[0]
        row_data[blank_bc_idx] = blank_barcode()
        try:
            row_data =  list(map(lambda x: str(x), fi_content_df.iloc[row_idx[i]].values.tolist())) + gil_num 
            first_4_vals = [str(nbr), datetime.now().strftime("%Y-%b-%d %H:%M:%S"), projectid, plateid_list[i]]
            row_data = first_4_vals + row_data
        except IndexError as e:
            pass
        if not commit:
            print(row_data)

        #print(all([type(x) == str for x in row_data]))
        #print(tuple(row_data))
        if commit:
            try:
                con.execute("INSERT INTO GILSON_TSL_TABLE VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row_data)
            except:
                print(row_data)
    if commit:
        con.commit()

    #return df_list

In [10]:
def grab_rows(fi_content_df):
    '''Grab the valid rows from the worklist file'''
    row_idx=[]
    for i in range(fi_content_df.shape[0]):
        try:
            if pd.isna(fi_content_df['#Plate_Sample[True,115,13]'][i]):
                pass#print('nothing')
            
            else:
                row_idx.append(i)
        except TypeError:
            print(i)
    return row_idx
    
def blank_barcode():
    '''Create blank barcodes to satisfy the database integrity'''
    blank_bc =  "BLANK_" + ''.join(choice(characters) for x in range(randint(8, 16)))
    return blank_bc 


def get_col_names_tbl(table_name, con):
    '''get length of column names and content with indices; returns dictionary'''
    cursor = con.execute(f'SELECT * FROM {table_name}')
    names = list(map(lambda x: x[0], cursor.description))
    return {'length':len(names),'names':[(i,n) for i,n in enumerate(names)]}
    
    
def grab_colm_idx(col_name, con):
    '''returns just the column index that matches the query name'''
    return [i for i,n in get_col_names_tbl("GILSON_TSL_TABLE", con)['names'] if n == col_name][0]
    
def split_string_input_file(input_file):
    '''handles if contains filepath separator'''
    if '/' in input_file:
        input_file = input_file.split('/')[-1]
    return input_file


def imbue_rows(dir_fi, con, commit=False, gil_num=[gethostname()]):
    '''print out each row after extracting data from raw worklist file and then repare for sql entry'''
    nbr = 0 
    # check to see if the file path has '/'
    tmp_dir_fi = split_string_input_file(dir_fi)
    split_fi_name = tmp_dir_fi.split('_')
    projectid = split_fi_name[0]
    # read into pandas df and coerce datatypes to string
    fi_content_df = pd.read_csv(dir_fi, sep='\t',dtype={'NOTES_STRING[False,68,9]': str, \
                                                       'SampleId[False,90,5]' : str})
    # ensure strings for each value
    fi_content_df['SampleId[False,90,5]'] = fi_content_df['SampleId[False,90,5]'].apply(lambda x: 'NaN')
    fi_content_df['SampleAmount[False,78,3]'] = fi_content_df['SampleAmount[False,78,3]'].apply(lambda x: f"{str(x)}")
    fi_content_df['#Sample Well[True,109,10]'] = fi_content_df['#Sample Well[True,109,10]'].apply(lambda x: f"{str(x)}")
    # split string and create new column
    fi_content_df['plateid'] = fi_content_df['NOTES_STRING[False,68,9]'].str.split('/').str[1]
    # drop the original column since we got the projecti from the file already
    fi_content_df.drop('NOTES_STRING[False,68,9]', axis=1, inplace=True)
    current_col_names = fi_content_df.columns.tolist()[:-1]
    # remove plateid colm as separate list
    plateid_list = [x for x in fi_content_df.plateid.tolist() if not pd.isna(x)]
    # slice df to exlucde plateid
    fi_content_df = fi_content_df[current_col_names]
    
    # get the valid row indices (non-flush/std/shutdown)
    row_idx = grab_rows(fi_content_df)
    len_rows = len(row_idx)
    # get length of gilson table to import to
    len_cols = get_col_names_tbl('GILSON_TSL_TABLE', con)['length']
    # if not a full set, add blanks 
    if len_rows % 4 != 0:
        len_rows = len_rows + (4-len_rows%4)
    # the index nbr for the blank sample well
    blank_sample_idx = grab_colm_idx('SAMPLE_WELL', con) 
    # add random blank barcode
    blank_bc_idx = grab_colm_idx('SAMPLE_DESCRIPTION', con) 
    
    for i in range(len_rows): 
        nbr += 1
        row_data = ["BLANK"]*len_cols
        row_data[blank_sample_idx] = nbr
        row_data[2] = projectid
        row_data[0] = nbr
        row_data[-1] = gil_num[0]
        row_data[blank_bc_idx] = blank_barcode()
        print(row_data)
        try:
            row_data =  list(map(lambda x: str(x), fi_content_df.iloc[row_idx[i]].values.tolist())) + gil_num 
            first_4_vals = [str(nbr), datetime.now().strftime("%Y-%b-%d %H:%M:%S"), projectid, plateid_list[i]]
            row_data = first_4_vals + row_data
        except IndexError as e:
             print('passing...')
        print(row_data)

In [45]:
characters = string.ascii_letters + string.punctuation  + string.digits

In [46]:
imbue_rows(f'test_files/1520060001_secStg_15200600010_edit.tsl', con)

[1, 'BLANK', '1520060001', 'BLANK', 'BLANK', 'BLANK', 'BLANK', "BLANK_IKA'P_T>!REY7iUl", 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 1, 'BLANK', 'FR-N-S081093']
['1', '2020-Jun-26 17:52:21', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L129525_6', '0', 'FB04587339', 'NaN', 'YES', 'UNKNOWN', '...', '1', 'P01S01', 'FR-N-S081093']
[2, 'BLANK', '1520060001', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK_[#(xm1l"pu*nAM%-', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 2, 'BLANK', 'FR-N-S081093']
['2', '2020-Jun-26 17:52:21', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L48809_5', '0', 'FB04251130', 'NaN', 'YES', 'UNKNOWN', '...', '2', 'P01S02', 'FR-N-S081093']
[3, 'BLANK', '1520060001', 'BLANK', 'BLANK', 'BLANK', 'BLANK', "BLANK_p+mi=/QjV'\\/nIC", 'BLANK', 'BLANK', 'BLANK', 'BLANK', 'BLANK', 3, 'BLANK', 'FR-N-S081093']
['3', '2020-Jun-26 17:52:21', '1520060001', ' 11', 'RUN', '4ML_10mm_2ND STAGE', 'L129575_6', '0', 'FB04716251', 'NaN', 'YES', 'UNKNOWN', '...', '3', 'P01S03', 'FR-N-

In [4]:
ORACLE_COLM_NAMES = ['SEQ_NUM',
                    'FINISH_DATE',
                    'PROJECT_ID',
                    'METHOD_NAME',
                    'SAMPLE_NAME',
                    'BARCODE',
                    'PLATE_ID',
                    'BROOKS_BARCODE',
                    'SAMPLE_WELL',
                    'PLATE_POSITION',
                    'GILSON_NUMBER']

In [129]:

df = pd.read_csv('test_files/1520030001_secStg_127.tsl', sep='\t',dtype={'NOTES_STRING[False,68,9]': str, \
                                                       'SampleId[False,90,5]' : str})
df.head()

,"SkipPause[True,18,0]","MethodName[True,239,1]","SampleName[True,108,2]","SampleAmount[False,78,3]","SampleDescription[False,120,4]","SampleId[False,90,5]","InjectionYesNo[False,72,6]","PeakInformation[False,104,7]","SampleInjection[False,138,8]","NOTES_STRING[False,68,9]","#Sample Well[True,109,10]","#Plate_Sample[True,115,13]"
0,RUN,4 ML STARTUP,Sample,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
1,RUN,4ML_10mm_2ND STAGE_FLUSH,Flush,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
2,RUN,4ML_10mm_2ND STAGE_STD,6-Cmpd Std,0,NaN,NaN,YES,UNKNOWN,...,NaN,0,NaN
3,RUN,4ML_10mm_2ND STAGE,M21535_5,0,FB02646205,NaN,YES,UNKNOWN,...,127,1,P01S01
4,RUN,4ML_10mm_2ND STAGE,M21551_5,0,FB02646225,NaN,YES,UNKNOWN,...,127,2,P01S02


In [128]:
def imbue_rows(dir_fi, con, gil_num=[gethostname()]):
    '''print out each row after extracting data from raw worklist file and then repare for sql entry'''
    # check to see if the file path has '/'
    tmp_dir_fi = split_string_input_file(dir_fi)
    split_fi_name = tmp_dir_fi.split('_')
    projectid = split_fi_name[0]
    # read into pandas df and coerce datatypes to string
    fi_content_df = pd.read_csv(dir_fi, sep='\t',dtype={'NOTES_STRING[False,68,9]': str, \
                                                       'SampleId[False,90,5]' : str})
    # ensure strings for each value
    fi_content_df['SampleId[False,90,5]'] = fi_content_df['SampleId[False,90,5]'].apply(lambda x: 'NaN')
    fi_content_df['SampleAmount[False,78,3]'] = fi_content_df['SampleAmount[False,78,3]'].apply(lambda x: f"{str(x)}")
    fi_content_df['#Sample Well[True,109,10]'] = fi_content_df['#Sample Well[True,109,10]'].apply(lambda x: f"{str(x)}")
    # split string and create new column
    fi_content_df['plateid'] = fi_content_df['NOTES_STRING[False,68,9]'].apply(lambda x: x.split('/')[1].strip() if not pd.isna(x) else '0') 
    fi_content_df['brooks_barcode'] = fi_content_df['NOTES_STRING[False,68,9]'].apply(lambda x: x.split('/')[0].strip() if not pd.isna(x) else '0')
    
    # get the valid row indices (non-flush/std/shutdown)
    row_idx = grab_rows(fi_content_df)
    
    current_time =  datetime.now().strftime("%Y-%b-%d %H:%M")
    
    with con.cursor() as cursor:
        for i in range(24): 
            row_data = ["BLANK"]*11
            row_data[0] = i 
            row_data[1] = current_time 
            row_data[2] = projectid
            row_data[9] = i 
            row_data[10] = gil_num[0]
            try:
                rdata_df =  fi_content_df.iloc[row_idx[i]]
                row_data[3] = rdata_df['MethodName[True,239,1]']
                row_data[4] = rdata_df['SampleName[True,108,2]']
                row_data[5] = rdata_df['SampleDescription[False,120,4]']
                row_data[6] = rdata_df['plateid']
                row_data[7] = rdata_df['brooks_barcode']
                row_data[8] = str(int(float(rdata_df['#Sample Well[True,109,10]'])))
                row_data[9] = rdata_df['#Plate_Sample[True,115,13]']
            except IndexError: 
                 print('passing as a blank ...')
            #print(row_data)
            cursor.execute(f"""insert into GILSON_RUN_LOGS values (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11)""", row_data)
    con.commit()

In [126]:
imbue_rows('/Volumes/npsg/tecan/SourceData/SecondStage/Sample List_Combined_tmp/15200600_01_02_15200600010_15200600020.tsl', conn)